## Summary
- Baseline written using Pytorch Lightning
- resnest26d as encoder
- Dataset is taken from: https://www.kaggle.com/datasets/shashwatraman/contrails-images-ash-color

#### Improvements over [previous version](https://www.kaggle.com/code/egortrushin/gr-icrgw-pytorch-lightning-baseline-unet-resnest) (please upvote).
- Option to change image size
- Mixed precision training (only useful with T4x2, on P100 this slows down training). This helps to use GPU memory more efficiently
- Training using 2 GPUs - with 2 GPUs we have more memory and higher speed
- Other numerous small changes

### Training part

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torchvision import utils

from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional import dice
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import os
import time
import pandas as pd
from pprint import pprint
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from xgboost import XGBClassifier
import shap
from scipy.spatial import distance

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
"""
df = pd.read_csv('/kaggle/input/contourails-features/contourails_features.csv')
df = df.sample(len(df),random_state=786)
dfF = df.drop(["is_contrail"],axis=1)
dfL = df[['is_contrail']]
N = 6197
x_train_df_p=dfF.iloc[0:N]
y_train_df_p=dfL.iloc[0:N]
x_test_df_p=dfF.iloc[N:-1]
y_test_df_p=dfL.iloc[N:-1]
modelML = CatBoostClassifier(iterations=500,max_depth=5,logging_level='Silent') 
modelML.fit(x_train_df_p,y_train_df_p)
y_pred = modelML.predict(x_test_df_p)
print('Test Accuracy',accuracy_score(y_test_df_p, y_pred))
print('Test F1',f1_score(y_test_df_p, y_pred))
print('Test F0.5',fbeta_score(y_test_df_p, y_pred,beta=0.5))
print('Test Precision',precision_score(y_test_df_p, y_pred))
print('Test Recall',recall_score(y_test_df_p, y_pred))
"""

'\ndf = pd.read_csv(\'/kaggle/input/contourails-features/contourails_features.csv\')\ndf = df.sample(len(df),random_state=786)\ndfF = df.drop(["is_contrail"],axis=1)\ndfL = df[[\'is_contrail\']]\nN = 6197\nx_train_df_p=dfF.iloc[0:N]\ny_train_df_p=dfL.iloc[0:N]\nx_test_df_p=dfF.iloc[N:-1]\ny_test_df_p=dfL.iloc[N:-1]\nmodelML = CatBoostClassifier(iterations=500,max_depth=5,logging_level=\'Silent\') \nmodelML.fit(x_train_df_p,y_train_df_p)\ny_pred = modelML.predict(x_test_df_p)\nprint(\'Test Accuracy\',accuracy_score(y_test_df_p, y_pred))\nprint(\'Test F1\',f1_score(y_test_df_p, y_pred))\nprint(\'Test F0.5\',fbeta_score(y_test_df_p, y_pred,beta=0.5))\nprint(\'Test Precision\',precision_score(y_test_df_p, y_pred))\nprint(\'Test Recall\',recall_score(y_test_df_p, y_pred))\n'

In [3]:
import sys
sys.path.append("../input/pretrained-models-pytorch")
sys.path.append("../input/efficientnet-pytorch")
sys.path.append("/kaggle/input/smp-github/segmentation_models.pytorch-master")
sys.path.append("/kaggle/input/timm-pretrained-resnest/resnest/")
import segmentation_models_pytorch as smp

"is" with a literal. Did you mean "=="?
"is" with a literal. Did you mean "=="?
"is" with a literal. Did you mean "=="?


In [4]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/timm-pretrained-resnest/resnest/gluon_resnest26-50eb607c.pth /root/.cache/torch/hub/checkpoints/gluon_resnest26-50eb607c.pth

In [5]:
#26 #26 epochs

In [6]:
from typing import Optional, List

import torch
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

#from ._functional import soft_dice_score, to_tensor


def to_tensor(x, dtype=None) -> torch.Tensor:
    if isinstance(x, torch.Tensor):
        if dtype is not None:
            x = x.type(dtype)
        return x
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x)
        if dtype is not None:
            x = x.type(dtype)
        return x
    if isinstance(x, (list, tuple)):
        x = np.array(x)
        x = torch.from_numpy(x)
        if dtype is not None:
            x = x.type(dtype)
        return x

def soft_dice_score(
    output: torch.Tensor,
    target: torch.Tensor,
    smooth: float = 0.0,
    eps: float = 1e-7,
    dims=None,
) -> torch.Tensor:
    assert output.size() == target.size()
    if dims is not None:
        intersection = torch.sum(output * target, dim=dims)
        cardinality = torch.sum(output + target, dim=dims)
    else:
        intersection = torch.sum(output * target)
        cardinality = torch.sum(output + target)
    #dice_score = (2.0 * intersection + smooth) / (cardinality + smooth).clamp_min(eps)
    dice_score = (100.0 * intersection + smooth) / (cardinality + smooth).clamp_min(eps)
    return dice_score


BINARY_MODE: str = "binary"
MULTICLASS_MODE: str = "multiclass"
MULTILABEL_MODE: str = "multilabel"

__all__ = ["DiceLoss"]


class DiceLoss(_Loss):
    def __init__(
        self,
        mode: str,
        classes: Optional[List[int]] = None,
        log_loss: bool = False,
        from_logits: bool = True,
        smooth: float = 0.0,
        ignore_index: Optional[int] = None,
        eps: float = 1e-7,
    ):

        assert mode in {BINARY_MODE, MULTILABEL_MODE, MULTICLASS_MODE}
        super(DiceLoss, self).__init__()
        self.mode = mode
        if classes is not None:
            assert mode != BINARY_MODE, "Masking classes is not supported with mode=binary"
            classes = to_tensor(classes, dtype=torch.long)

        self.classes = classes
        self.from_logits = from_logits
        self.smooth = smooth
        self.eps = eps
        self.log_loss = log_loss
        self.ignore_index = ignore_index

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:

        assert y_true.size(0) == y_pred.size(0)

        if self.from_logits:
            # Apply activations to get [0..1] class probabilities
            # Using Log-Exp as this gives more numerically stable result and does not cause vanishing gradient on
            # extreme values 0 and 1
            if self.mode == MULTICLASS_MODE:
                y_pred = y_pred.log_softmax(dim=1).exp()
            else:
                y_pred = F.logsigmoid(y_pred).exp()

        bs = y_true.size(0)
        num_classes = y_pred.size(1)
        dims = (0, 2)

        if self.mode == BINARY_MODE:
            y_true = y_true.view(bs, 1, -1)
            y_pred = y_pred.view(bs, 1, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask
                y_true = y_true * mask

        if self.mode == MULTICLASS_MODE:
            y_true = y_true.view(bs, -1)
            y_pred = y_pred.view(bs, num_classes, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask.unsqueeze(1)

                y_true = F.one_hot((y_true * mask).to(torch.long), num_classes)  # N,H*W -> N,H*W, C
                y_true = y_true.permute(0, 2, 1) * mask.unsqueeze(1)  # N, C, H*W
            else:
                y_true = F.one_hot(y_true, num_classes)  # N,H*W -> N,H*W, C
                y_true = y_true.permute(0, 2, 1)  # N, C, H*W

        if self.mode == MULTILABEL_MODE:
            y_true = y_true.view(bs, num_classes, -1)
            y_pred = y_pred.view(bs, num_classes, -1)

            if self.ignore_index is not None:
                mask = y_true != self.ignore_index
                y_pred = y_pred * mask
                y_true = y_true * mask

        scores = self.compute_score(y_pred, y_true.type_as(y_pred), smooth=self.smooth, eps=self.eps, dims=dims)

        if self.log_loss:
            loss = -torch.log(scores.clamp_min(self.eps))
        else:
            loss = 1.0 - scores

        # Dice loss is undefined for non-empty classes
        # So we zero contribution of channel that does not have true pixels
        # NOTE: A better workaround would be to use loss term `mean(y_pred)`
        # for this case, however it will be a modified jaccard loss

        mask = y_true.sum(dims) > 0
        loss *= mask.to(loss.dtype)

        if self.classes is not None:
            loss = loss[self.classes]

        return self.aggregate_loss(loss)

    def aggregate_loss(self, loss):
        return loss.mean()

    def compute_score(self, output, target, smooth=0.0, eps=1e-7, dims=None) -> torch.Tensor:
        return soft_dice_score(output, target, smooth, eps, dims)


In [7]:
%%writefile config.yaml

data_path: "/kaggle/input/contrails-images-ash-color"
output_dir: "models"

seed: 42

train_bs: 32
valid_bs: 128
workers: 2

progress_bar_refresh_rate: 1

early_stop:
    monitor: "val_loss"
    mode: "min"
    patience: 999
    verbose: 1

trainer:
    max_epochs: 26
    min_epochs: 26
    enable_progress_bar: True
    precision: "16-mixed"
    devices: 2

model:
    seg_model: "Unet"
    encoder_name: "timm-resnest26d"
    loss_smooth: 1.0
    image_size: 384
    optimizer_params:
        lr: 0.00085
        weight_decay: 0.0
    scheduler:
        name: "CosineAnnealingLR"
        params:
            CosineAnnealingLR:
                T_max: 2
                eta_min: 1.0e-6
                last_epoch: -1
            ReduceLROnPlateau:
                mode: "min"
                factor: 0.31622776601
                patience: 4
                verbose: True

Writing config.yaml


In [8]:
#26 26 0.0005
#T_max: 2
#eta_min: 1.0e-6
# 48
#0.31622776601

In [9]:
# Dataset

import torch
import numpy as np
import torchvision.transforms as T

class ContrailsDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_size=256, train=True):

        self.df = df
        self.trn = train
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.image_size = image_size
        if image_size != 256:
            self.resize_image = T.transforms.Resize(image_size)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        con = np.load(str(con_path))

        img = con[..., :-1]
        label = con[..., -1]

        label = torch.tensor(label)

        img = torch.tensor(np.reshape(img, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)

        if self.image_size != 256:
            img = self.resize_image(img)

        img = self.normalize_image(img)

        return img.float(), label.float()

    def __len__(self):
        return len(self.df)

In [10]:
# Lightning module

import torch
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn as nn
from torchmetrics.functional import dice

seg_models = {
    "Unet": smp.Unet,
    "Unet++": smp.UnetPlusPlus,
    "MAnet": smp.MAnet,
    "Linknet": smp.Linknet,
    "FPN": smp.FPN,
    "PSPNet": smp.PSPNet,
    "PAN": smp.PAN,
    "DeepLabV3": smp.DeepLabV3,
    "DeepLabV3+": smp.DeepLabV3Plus,
}


class LightningModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = model = seg_models[config["seg_model"]](
            encoder_name=config["encoder_name"],
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            activation=None,
        )
        
        #self.loss_module = smp.losses.DiceLoss(mode="binary", smooth=config["loss_smooth"])
        self.loss_module = DiceLoss(mode="binary", smooth=config["loss_smooth"])
        
        
        self.val_step_outputs = []
        self.val_step_labels = []

    def forward(self, batch):
        imgs = batch
        preds = self.model(imgs)
        return preds

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config["optimizer_params"])

        if self.config["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config["scheduler"]["params"]["CosineAnnealingLR"],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config["scheduler"]["params"]["ReduceLROnPlateau"],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        if self.config["image_size"] != 256:
            preds = torch.nn.functional.interpolate(preds, size=256, mode='bilinear')
        loss = self.loss_module(preds, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=16)

        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=False, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        if self.config["image_size"] != 256:
            preds = torch.nn.functional.interpolate(preds, size=256, mode='bilinear')
        loss = self.loss_module(preds, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.val_step_outputs.append(preds)
        self.val_step_labels.append(labels)

    def on_validation_epoch_end(self):
        all_preds = torch.cat(self.val_step_outputs)
        all_labels = torch.cat(self.val_step_labels)
        all_preds = torch.sigmoid(all_preds)
        self.val_step_outputs.clear()
        self.val_step_labels.clear()
        val_dice = dice(all_preds, all_labels.long())
        self.log("val_dice", val_dice, on_step=False, on_epoch=True, prog_bar=True)
        if self.trainer.global_rank == 0:
            print(f"\nEpoch: {self.current_epoch}", flush=True)

In [11]:
# Actual training

import warnings

warnings.filterwarnings("ignore")

import os
import torch
import yaml
import pandas as pd
import pytorch_lightning as pl
from pprint import pprint
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.utils.data import DataLoader

with open("config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)

contrails = os.path.join(config["data_path"], "contrails/")
train_path = os.path.join(config["data_path"], "train_df.csv")
valid_path = os.path.join(config["data_path"], "valid_df.csv")

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

train_df["path"] = contrails + train_df["record_id"].astype(str) + ".npy"
valid_df["path"] = contrails + valid_df["record_id"].astype(str) + ".npy"

dataset_train = ContrailsDataset(train_df, config["model"]["image_size"], train=True)
dataset_validation = ContrailsDataset(valid_df, config["model"]["image_size"], train=False)

data_loader_train = DataLoader(
    dataset_train,
    batch_size=config["train_bs"],
    shuffle=True,
    num_workers=config["workers"],
)
data_loader_validation = DataLoader(
    dataset_validation,
    batch_size=config["valid_bs"],
    shuffle=False,
    num_workers=config["workers"],
)

checkpoint_callback = ModelCheckpoint(
    save_weights_only=True,
    monitor="val_dice",
    dirpath=config["output_dir"],
    mode="max",
    filename="model",
    save_top_k=1,
    verbose=1,
)

progress_bar_callback = TQDMProgressBar(
    refresh_rate=config["progress_bar_refresh_rate"]
)

early_stop_callback = EarlyStopping(**config["early_stop"])

trainer = pl.Trainer(
    callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback],
    **config["trainer"],
)

config["model"]["scheduler"]["params"]["CosineAnnealingLR"]["T_max"] *= len(data_loader_train)/config["trainer"]["devices"]
model = LightningModule(config["model"])



In [12]:
trainer.fit(model, data_loader_train, data_loader_validation)


Epoch: 0


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]


Epoch: 0


Validation: 0it [00:00, ?it/s]

...


Epoch: 23


Validation: 0it [00:00, ?it/s]


Epoch: 24


Validation: 0it [00:00, ?it/s]


Epoch: 25


In [13]:
def displayImageSRGB(imgI,imgname):
    if imgI.shape[0]>0:
        f = plt.figure()
        f.set_figwidth(3)
        f.set_figheight(3)
        #rgb = cv2.cvtColor(imgI, cv2.COLOR_BGR2RGB)
        plt.imshow( imgI  )
        plt.title(imgname)
        plt.show()
        
def displayImageSBGR(imgI,imgname):
    if imgI.shape[0]>0:
        f = plt.figure()
        f.set_figwidth(5)
        f.set_figheight(5)
        rgb = cv2.cvtColor(imgI, cv2.COLOR_BGR2RGB)
        plt.imshow( rgb  )
        plt.title(imgname)
        plt.show()


def displayImageGreyS(imgI,imgname):
    if imgI.shape[0]>0:
        f = plt.figure()
        f.set_figwidth(3)
        f.set_figheight(3)
        #plt.imshow( imgI, cmap='gray' )
        plt.imshow( imgI)
        plt.title(imgname)
        plt.show()

def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

def get_id_images(record_id):
    BASE_DIR = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation'
    N_TIMES_BEFORE = 4

    with open(os.path.join(BASE_DIR, record_id, 'band_11.npy'), 'rb') as f:
        band11 = np.load(f)
    with open(os.path.join(BASE_DIR, record_id, 'band_14.npy'), 'rb') as f:
        band14 = np.load(f)
    with open(os.path.join(BASE_DIR, record_id, 'band_15.npy'), 'rb') as f:
        band15 = np.load(f)
    with open(os.path.join(BASE_DIR, record_id, 'human_pixel_masks.npy'), 'rb') as f:
        human_pixel_mask = np.load(f)
    #with open(os.path.join(BASE_DIR, record_id, 'human_individual_masks.npy'), 'rb') as f:
    #    human_individual_mask = np.load(f)
        
    _T11_BOUNDS = (243, 303)
    _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
    _TDIFF_BOUNDS = (-4, 2)
    
    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    
    rgb_images = []
    for t in range(8):
        rgb_images.append( false_color[..., t ] )
        
    return (false_color,rgb_images,human_pixel_mask)


def get_id_imagesTest(record_id):
    BASE_DIR = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test'
    N_TIMES_BEFORE = 4

    with open(os.path.join(BASE_DIR, record_id, 'band_11.npy'), 'rb') as f:
        band11 = np.load(f)
    with open(os.path.join(BASE_DIR, record_id, 'band_14.npy'), 'rb') as f:
        band14 = np.load(f)
    with open(os.path.join(BASE_DIR, record_id, 'band_15.npy'), 'rb') as f:
        band15 = np.load(f)
        
    _T11_BOUNDS = (243, 303)
    _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
    _TDIFF_BOUNDS = (-4, 2)
    
    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    
    rgb_images = []
    for t in range(8):
        rgb_images.append( false_color[..., t ] )
        
    return (false_color,rgb_images)



def getContourPixels(contour,img,imgRGB):
    lst_intensities = []
    lst_rvals = []
    lst_gvals = []
    lst_bvals = []
    lst_positions = []
    cimg = np.zeros_like(img)
    cv2.drawContours(cimg, [contour], 0, color=255, thickness=-1)
    pts = np.where(cimg == 255)
    for ip in range(len(pts[0])):
        #x = pts[0][ip]
        #y = pts[1][ip]
        y = pts[0][ip]
        x = pts[1][ip]
        lst_positions.append( (x,y) )
        lst_intensities.append( img[y,x]  )
        lst_rvals.append( imgRGB[y,x,0] )
        lst_gvals.append( imgRGB[y,x,1] )
        lst_bvals.append( imgRGB[y,x,2] )
    return lst_positions,lst_intensities,lst_rvals,lst_gvals,lst_bvals


def getPrependicularIntersector_Single(lineSeg,prepDist,offset1,imgH):
    
        x1 = float(lineSeg[0][0])
        y1 = float(imgH-lineSeg[0][1])
        x2 = float(lineSeg[1][0])
        y2 = float(imgH-lineSeg[1][1])
        N = prepDist

        dx = x1-x2
        dy = y1-y2
        dist = distance.euclidean((x1,y1), (x2,y2))
        dx /= dist
        dy /= dist

        posVec = [x2-x1,y2-y1]
        V = np.array( posVec )
        mag = np.linalg.norm(V)
        unitV = V / mag

        offsetVec1 = unitV * offset1

        xo1 = x1 + offsetVec1[0]
        yo1 = y1 + offsetVec1[1]

        x3 = xo1 + (N/2)*dy
        y3 = yo1 - (N/2)*dx
        x4 = xo1 - (N/2)*dy
        y4 = yo1 + (N/2)*dx
        y4 = imgH-y4
        y3 = imgH-y3

        P1 = [  [round(x3),round(y3)],[ round(x4),round(y4) ] ]

        return P1
    
    
def get_contourail_features(contour,gray_img,img_uint):
    #color = (np.random.randint(100, 200), np.random.randint(100, 200), np.random.randint(100, 200))
    lst_positions,lst_intensities,lst_rvals,lst_gvals,lst_bvals = getContourPixels(contour, gray_img, img_uint)

    median_inten = np.median(lst_intensities)
    std_inten = np.std(lst_intensities)
    median_r = np.median(lst_rvals)
    median_g = np.median(lst_gvals)
    median_b = np.median(lst_bvals)

    area = cv2.contourArea(contour)
    x,y,w,h = cv2.boundingRect(contour)
    rect_area = w*h
    extent = float(area)/rect_area #####
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    box_pts = [ list(box[0]),list(box[1]),list(box[2]),list(box[3]) ]
    #cv2.drawContours(bgr_img1,[box],0,(255,0,0),1)
    dist_list = []
    for boi in range(4):
        for boj in range(4):
            if boi != boj:
                bxi,byi = box_pts[boi][0],box_pts[boi][1] 
                bxj,byj = box_pts[boj][0],box_pts[boj][1] 
                dist = distance.euclidean((bxi,byi), (bxj,byj))
                dist_list.append( ((bxi,byi),(bxj,byj),dist)  )

    dist_list.sort(key = lambda lx: lx[2])
    dist_list2 = []
    for di,dtup in enumerate(dist_list):
        if di%2==0:
            dist_list2.append(dtup)

    bbL = dist_list2[3][2]
    bbW = dist_list2[1][2]
    if bbW != 0:
        bbLoW = bbL/bbW
    else:
        bbLoW = 1000

    L1 = tuple(dist_list2[3][:-1])
    L2 = tuple(dist_list2[2][:-1])
    if distance.euclidean(L1[0], L2[0]) > distance.euclidean(L1[0], L2[1]):
        L2 = ( L2[1],L2[0] )
    L3 = [  ( int((L1[0][0]+L2[0][0])/2) ,int((L1[0][1]+L2[0][1])/2)  ),( int((L1[1][0]+L2[1][0])/2) ,int((L1[1][1]+L2[1][1])/2) )    ]

    #cv2.line(bgr_img1,L3[0],L3[1],(255,0,255),1)
    lineLen = distance.euclidean(L3[0],L3[1])
    prep_points = []
    for lp in [0.16,0.33,0.5,0.66,0.83]:
        prepdist = int(lineLen*lp)
        PRI = getPrependicularIntersector_Single(L3,11,prepdist,gray_img.shape[0])
        PP1 = PRI[0]
        PP2 = PRI[1]
        prep_points.append(PP1)
        prep_points.append(PP2)
    off_intens = []
    off_rgbs = []
    for prpt in prep_points:
        px = prpt[0]
        py = prpt[1]
        if px>= 0 and px<gray_img.shape[1] and py>=0 and py<gray_img.shape[0]:
            off_intens.append( gray_img[py,px]  )
            off_rgbs.append( tuple(img_uint[py,px])  )

            #bgr_img[py,px] = (255,255,0)

    #print(off_intens)
    #print(off_rgbs)

    median_inten_off = np.median(off_intens)
    std_inten_off = np.std(off_intens)
    inten_onoff_diff = abs(median_inten_off-median_inten)

    median_r_off = np.median( [ v[0] for v in off_rgbs ]  )
    median_g_off = np.median( [ v[1] for v in off_rgbs ]  )
    median_b_off = np.median( [ v[2] for v in off_rgbs ]  )

    r_onoff_diff = abs(median_r_off-median_r)
    g_onoff_diff = abs(median_g_off-median_g)
    b_onoff_diff = abs(median_b_off-median_b)

    rgb_diff = distance.euclidean((median_r,median_g,median_b), (median_r_off,median_g_off,median_b_off) )

    return (median_inten,std_inten,median_r,median_g,median_b,median_r_off,median_g_off,median_b_off,std_inten_off,area,extent,bbL,bbW,bbLoW,median_inten_off,inten_onoff_diff,r_onoff_diff,g_onoff_diff,b_onoff_diff,rgb_diff)

feats_list = ['median_inten','std_inten','median_r','median_g','median_b','median_r_off','median_g_off','median_b_off','std_inten_off','area','extent','bbL','bbW','bbLoW','median_inten_off','inten_onoff_diff','r_onoff_diff','g_onoff_diff','b_onoff_diff','rgb_diff']

### Submission part

In [14]:
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'
data_root = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test/'

In [15]:
filenames = os.listdir(data_root)
test_df = pd.DataFrame(filenames, columns=['record_id'])
test_df['path'] = data_root + test_df['record_id'].astype(str)

In [16]:
class ContrailsDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_size=256, train=True):
        
        self.df = df
        self.trn = train
        self.df_idx: pd.DataFrame = pd.DataFrame({'idx': os.listdir(f'/kaggle/input/google-research-identify-contrails-reduce-global-warming/test')})
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.image_size = image_size
        if image_size != 256:
            self.resize_image = T.transforms.Resize(image_size)
    
    def read_record(self, directory):
        record_data = {}
        for x in [
            "band_11", 
            "band_14", 
            "band_15"
        ]:

            record_data[x] = np.load(os.path.join(directory, x + ".npy"))

        return record_data

    def normalize_range(self, data, bounds):
        """Maps data to the range [0, 1]."""
        return (data - bounds[0]) / (bounds[1] - bounds[0])
    
    def get_false_color(self, record_data):
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)
        
        N_TIMES_BEFORE = 4

        r = self.normalize_range(record_data["band_15"] - record_data["band_14"], _TDIFF_BOUNDS)
        g = self.normalize_range(record_data["band_14"] - record_data["band_11"], _CLOUD_TOP_TDIFF_BOUNDS)
        b = self.normalize_range(record_data["band_14"], _T11_BOUNDS)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        img = false_color[..., N_TIMES_BEFORE]

        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        data = self.read_record(con_path)    
        
        img = self.get_false_color(data)
        
        img = torch.tensor(np.reshape(img, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)
        
        if self.image_size != 256:
            img = self.resize_image(img)
        
        img = self.normalize_image(img)
        
        image_id = int(self.df_idx.iloc[index]['idx'])
            
        return img.float(), torch.tensor(image_id)
    
    def __len__(self):
        return len(self.df)

In [17]:
test_ds = ContrailsDataset(
        test_df,
        config["model"]["image_size"],
        train = False
    )
 
test_dl = DataLoader(test_ds, batch_size=batch_size, num_workers = 1)

In [18]:
class LightningModule(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = smp.Unet(encoder_name="timm-resnest26d",
                              encoder_weights=None,
                              in_channels=3,
                              classes=1,
                              activation=None,
                              )

    def forward(self, batch):
        return self.model(batch)

In [19]:
model = LightningModule().load_from_checkpoint("/kaggle/working/models/model.ckpt")
#model = LightningModule().load_from_checkpoint("/kaggle/input/unet-highrecalllossfunction-models/model10R.ckpt")
#model = LightningModule().load_from_checkpoint("/kaggle/input/unet-model12-contrails/model2.ckpt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
model.zero_grad()

In [20]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

In [21]:
def getContourPixels(contour,img,imgRGB):
    lst_intensities = []
    lst_rvals = []
    lst_gvals = []
    lst_bvals = []
    lst_positions = []
    cimg = np.zeros_like(img)
    cv2.drawContours(cimg, [contour], 0, color=255, thickness=-1)
    pts = np.where(cimg == 255)
    for ip in range(len(pts[0])):
        #x = pts[0][ip]
        #y = pts[1][ip]
        y = pts[0][ip]
        x = pts[1][ip]
        lst_positions.append( (x,y) )
        lst_intensities.append( img[y,x]  )
        lst_rvals.append( imgRGB[y,x,0] )
        lst_gvals.append( imgRGB[y,x,1] )
        lst_bvals.append( imgRGB[y,x,2] )
    return lst_positions,lst_intensities,lst_rvals,lst_gvals,lst_bvals

def checkIsContrail(lst_positions,gtmask,thresha=0.5):
    no_pos = len(lst_positions)
    corrects = 0
    for pos in lst_positions:
        x = pos[0]
        y = pos[1]
        if gtmask[y,x] > 0:
            corrects += 1
    frac = float(corrects) / float(no_pos)
    #print('frac',frac,'corrects',corrects)
    if frac > thresha:
        return 1
    else:
        return 0
    

def getLineScan(P1, P2, img):
        """
        Produces and array that consists of the coordinates and intensities of each pixel in a line between two points

        Parameters:
            -P1: a tuple that consists of the coordinate of the first point (x,y)
            -P2: a tuple that consists of the coordinate of the second point (x,y)
            -img: the image being processed

        Returns:
            -it: a numpy array that consists of the coordinates and intensities of each pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
        """
        P1 = np.array( [P1[0],P1[1]] )
        P2 = np.array( [P2[0],P2[1]] )
        #define local variables for readability
        imageH = img.shape[0]
        imageW = img.shape[1]
        P1X = P1[0]
        P1Y = P1[1]
        P2X = P2[0]
        P2Y = P2[1]

        #difference and absolute difference between points
        #used to calculate slope and relative location between points
        dX = P2X - P1X
        dY = P2Y - P1Y
        dXa = np.abs(dX)
        dYa = np.abs(dY)

        #predefine numpy array for output based on distance between points
        itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3),dtype=np.float32)
        itbuffer.fill(np.nan)

        #Obtain coordinates along the line using a form of Bresenham's algorithm
        negY = P1Y > P2Y
        negX = P1X > P2X
        if P1X == P2X: #vertical line segment
            itbuffer[:,0] = P1X
            if negY:
                itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
        elif P1Y == P2Y: #horizontal line segment
            itbuffer[:,1] = P1Y
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
        else: #diagonal line segment
            steepSlope = dYa > dXa
            if steepSlope:
                slope = dX.astype(np.float32)/dY.astype(np.float32)
                if negY:
                    itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
                else:
                    itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
                itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
            else:
                slope = dY.astype(np.float32)/dX.astype(np.float32)
                if negX:
                    itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
                else:
                    itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
                itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

        #Remove points outside of image
        
        colX = itbuffer[:,0]
        colY = itbuffer[:,1]
        itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<imageW) & (colY<imageH)]

        #Get intensities from img ndarray
        itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]

        return itbuffer
    
def getPrependicularIntersector_Single(lineSeg,prepDist,offset1,imgH):
    
        x1 = float(lineSeg[0][0])
        y1 = float(imgH-lineSeg[0][1])
        x2 = float(lineSeg[1][0])
        y2 = float(imgH-lineSeg[1][1])
        N = prepDist

        dx = x1-x2
        dy = y1-y2
        dist = distance.euclidean((x1,y1), (x2,y2))
        dx /= dist
        dy /= dist

        posVec = [x2-x1,y2-y1]
        V = np.array( posVec )
        mag = np.linalg.norm(V)
        unitV = V / mag

        offsetVec1 = unitV * offset1

        xo1 = x1 + offsetVec1[0]
        yo1 = y1 + offsetVec1[1]

        x3 = xo1 + (N/2)*dy
        y3 = yo1 - (N/2)*dx
        x4 = xo1 - (N/2)*dy
        y4 = yo1 + (N/2)*dx
        y4 = imgH-y4
        y3 = imgH-y3

        P1 = [  [round(x3),round(y3)],[ round(x4),round(y4) ] ]

        return P1
    
    
def get_contourail_features(contour,gray_img,img_uint,gtruth_mask,bgr_img):
    #color = (np.random.randint(100, 200), np.random.randint(100, 200), np.random.randint(100, 200))
    lst_positions,lst_intensities,lst_rvals,lst_gvals,lst_bvals = getContourPixels(contour, gray_img, img_uint)
    is_contrail = checkIsContrail(lst_positions,gtruth_mask)
    color = (0,0,255)
    if is_contrail==1:
        color = (0,255,0)
    #cv2.drawContours(bgr_img1, [contour], 0, color, 1)

    median_inten = np.median(lst_intensities)
    std_inten = np.std(lst_intensities)
    median_r = np.median(lst_rvals)
    median_g = np.median(lst_gvals)
    median_b = np.median(lst_bvals)

    area = cv2.contourArea(contour)
    x,y,w,h = cv2.boundingRect(contour)
    rect_area = w*h
    extent = float(area)/rect_area #####
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    box_pts = [ list(box[0]),list(box[1]),list(box[2]),list(box[3]) ]
    #cv2.drawContours(bgr_img1,[box],0,(255,0,0),1)
    dist_list = []
    for boi in range(4):
        for boj in range(4):
            if boi != boj:
                bxi,byi = box_pts[boi][0],box_pts[boi][1] 
                bxj,byj = box_pts[boj][0],box_pts[boj][1] 
                dist = distance.euclidean((bxi,byi), (bxj,byj))
                dist_list.append( ((bxi,byi),(bxj,byj),dist)  )

    dist_list.sort(key = lambda lx: lx[2])
    dist_list2 = []
    for di,dtup in enumerate(dist_list):
        if di%2==0:
            dist_list2.append(dtup)

    bbL = dist_list2[3][2]
    bbW = dist_list2[1][2]
    if bbW != 0:
        bbLoW = bbL/bbW
    else:
        bbLoW = 1000

    L1 = tuple(dist_list2[3][:-1])
    L2 = tuple(dist_list2[2][:-1])
    if distance.euclidean(L1[0], L2[0]) > distance.euclidean(L1[0], L2[1]):
        L2 = ( L2[1],L2[0] )
    L3 = [  ( int((L1[0][0]+L2[0][0])/2) ,int((L1[0][1]+L2[0][1])/2)  ),( int((L1[1][0]+L2[1][0])/2) ,int((L1[1][1]+L2[1][1])/2) )    ]

    #cv2.line(bgr_img1,L3[0],L3[1],(255,0,255),1)
    lineLen = distance.euclidean(L3[0],L3[1])
    prep_points = []
    for lp in [0.16,0.33,0.5,0.66,0.83]:
        prepdist = int(lineLen*lp)
        PRI = getPrependicularIntersector_Single(L3,11,prepdist,bgr_img.shape[0])
        PP1 = PRI[0]
        PP2 = PRI[1]
        prep_points.append(PP1)
        prep_points.append(PP2)
    off_intens = []
    off_rgbs = []
    for prpt in prep_points:
        px = prpt[0]
        py = prpt[1]
        if px>= 0 and px<bgr_img1.shape[1] and py>=0 and py<bgr_img1.shape[0]:
            off_intens.append( gray_img[py,px]  )
            off_rgbs.append( tuple(img_uint[py,px])  )

            #bgr_img[py,px] = (255,255,0)

    #print(off_intens)
    #print(off_rgbs)

    median_inten_off = np.median(off_intens)
    std_inten_off = np.std(off_intens)
    inten_onoff_diff = abs(median_inten_off-median_inten)

    median_r_off = np.median( [ v[0] for v in off_rgbs ]  )
    median_g_off = np.median( [ v[1] for v in off_rgbs ]  )
    median_b_off = np.median( [ v[2] for v in off_rgbs ]  )

    r_onoff_diff = abs(median_r_off-median_r)
    g_onoff_diff = abs(median_g_off-median_g)
    b_onoff_diff = abs(median_b_off-median_b)

    rgb_diff = distance.euclidean((median_r,median_g,median_b), (median_r_off,median_g_off,median_b_off) )

    return (median_inten,std_inten,median_r,median_g,median_b,median_r_off,median_g_off,median_b_off,std_inten_off,area,extent,bbL,bbW,bbLoW,median_inten_off,inten_onoff_diff,r_onoff_diff,g_onoff_diff,b_onoff_diff,rgb_diff,is_contrail)

feats_list = ['median_inten','std_inten','median_r','median_g','median_b','median_r_off','median_g_off','median_b_off','std_inten_off','area','extent','bbL','bbW','bbLoW','median_inten_off','inten_onoff_diff','r_onoff_diff','g_onoff_diff','b_onoff_diff','rgb_diff','is_contrail']

class ContrailsDatasetPLV(torch.utils.data.Dataset):
    def __init__(self, df, image_size=256, train=True):
        
        self.df = df
        self.trn = train
        self.df_idx: pd.DataFrame = pd.DataFrame({'idx': os.listdir(f'/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation')})
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.image_size = image_size
        if image_size != 256:
            self.resize_image = T.transforms.Resize(image_size)
    
    def read_record(self, directory):
        record_data = {}
        for x in [
            "band_11", 
            "band_14", 
            "band_15"
        ]:

            record_data[x] = np.load(os.path.join(directory, x + ".npy"))

        return record_data

    def normalize_range(self, data, bounds):
        """Maps data to the range [0, 1]."""
        return (data - bounds[0]) / (bounds[1] - bounds[0])
    
    def get_false_color(self, record_data):
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)
        
        N_TIMES_BEFORE = 4

        r = self.normalize_range(record_data["band_15"] - record_data["band_14"], _TDIFF_BOUNDS)
        g = self.normalize_range(record_data["band_14"] - record_data["band_11"], _CLOUD_TOP_TDIFF_BOUNDS)
        b = self.normalize_range(record_data["band_14"], _T11_BOUNDS)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        img = false_color[..., N_TIMES_BEFORE]

        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        data = self.read_record(con_path)    
        
        img = self.get_false_color(data)
        
        img = torch.tensor(np.reshape(img, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)
        
        if self.image_size != 256:
            img = self.resize_image(img)
        
        img = self.normalize_image(img)
        
        image_id = int(self.df_idx.iloc[index]['idx'])
            
        return img.float(), torch.tensor(image_id)
    
    def __len__(self):
        return len(self.df)
    
    
data_rootV = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation/'
lstdir = os.listdir(data_rootV)

image_size = 384 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
"""
import matplotlib.pyplot as plt
model.eval()

feats_dict = dict()
for feat in feats_list:
    feats_dict[feat] = []

def displayImageGreyS(imgI,imgname):
    if imgI.shape[0]>0:
        f = plt.figure()
        f.set_figwidth(3)
        f.set_figheight(3)
        plt.imshow( imgI, cmap='gray' )
        plt.title(imgname)
        plt.show()

ii = 0
#20
#while ii < len(lstdir)-2:
while ii < 20:
    print('ii',ii)
    #print(ii,'/',len(lstdir)-2)
    data_rootV = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation/'
    filenamesV = lstdir[ii:ii+2]
    ii += 2
    val_df = pd.DataFrame(filenamesV, columns=['record_id'])
    val_df['path'] = data_rootV + val_df['record_id'].astype(str)
    val_ds = ContrailsDatasetPLV(
        val_df,
        image_size,
        train = False
    )
    
    gtruth_masks = []
    
    for nn in range(len(val_df)) :
        sp = val_df['path'].iloc[nn]
        #print(sp)
        #print(sp+'/human_pixel_masks.npy')
        #print( sp[ sp.find('ion')+4 : ] )
        gtruth_masks.append( np.load( sp+'/human_pixel_masks.npy'   ) )
 
    val_dl = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers = 1)
    
    for i, data in enumerate(val_dl):
        images, image_id = data
        
        #im_id1 = str(image_id[0]).replace('tensor(','').replace(')','')
        #im_id2 = str(image_id[1]).replace('tensor(','').replace(')','')
        im_id1 = val_df['path'].iloc[0]
        im_id2 = val_df['path'].iloc[1]
        #print(im_id1,im_id2)
        
        false_color,rgb_images,human_pixel_mask = get_id_images(im_id1)
        img = rgb_images[4]
        img_uint1 = (img*255).astype(np.uint8)
        gray_img1 = cv2.cvtColor(img_uint1, cv2.COLOR_BGR2GRAY)
        
        false_color,rgb_images,human_pixel_mask = get_id_images(im_id2)
        img = rgb_images[4]
        img_uint2 = (img*255).astype(np.uint8)
        gray_img2 = cv2.cvtColor(img_uint2, cv2.COLOR_BGR2GRAY)
        
        images = images.to(DEVICE)
        with torch.no_grad():
            predicted_mask = model.forward(images[:, :, :, :])
        if image_size != 256:
            predicted_mask = torch.nn.functional.interpolate(predicted_mask, size=256, mode='bilinear')

         
        #imi
        predicted_mask = torch.sigmoid(predicted_mask).cpu().detach().numpy()
        #predicted_mask = predicted_mask.cpu().detach().numpy()
        
        
        
        #X=plt.hist((predicted_mask[0, 0, :, :]*255).astype(np.uint8))

        predicted_mask_with_threshold = np.zeros((images.shape[0], 256, 256))
        predicted_mask_with_threshold[predicted_mask[:, 0, :, :] < 0.05] = 0
        predicted_mask_with_threshold[predicted_mask[:, 0, :, :] > 0.05] = 1
        
        pred_img1 = (predicted_mask[0, 0, :, :]*255).astype(np.uint8)
        pred_img2 = (predicted_mask[1, 0, :, :]*255).astype(np.uint8)
        
        pred_img1F = predicted_mask[0, 0, :, :]
        pred_img2F = predicted_mask[1, 0, :, :]
        
        #edges1 = cv2.Canny(pred_img1, 10, 240)
        #edges1 = pred_img1
        edges1 = (predicted_mask_with_threshold[0,:,:]*255).astype(np.uint8)
        
        #thresh1, im_bw2 = cv2.threshold(pred_img1, 127, 255, cv2.THRESH_BINARY)
        #thresh2, im_bw2 = cv2.threshold(pred_img2, 127, 255, cv2.THRESH_BINARY)
        
        contours1, hierarchy1 = cv2.findContours(edges1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        #contours1, hierarchy1 = cv2.findContours(edges1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        #image = np.zeros((height,width,3), np.uint8)
        
        #bgr_img1 = cv2.cvtColor(pred_img1, cv2.CV_GRAY2BGR)
        #bgr_img2 = cv2.cvtColor(pred_img2, cv2.CV_GRAY2BGR)
        
        #X=plt.hist(pred_img1)
        
        bgr_img1 = np.zeros((pred_img1.shape[0], pred_img1.shape[1], 3), dtype=np.uint8)
        bgr_img1[(predicted_mask_with_threshold[0,:,:]*255).astype(np.uint8) == 255] = (255, 255, 255)
        
        bgr_img2 = np.zeros((pred_img2.shape[0], pred_img2.shape[1], 3), dtype=np.uint8)
        bgr_img2[(predicted_mask_with_threshold[1,:,:]*255).astype(np.uint8) == 255] = (255, 255, 255)
        
        edges2 = (predicted_mask_with_threshold[1,:,:]*255).astype(np.uint8)
        
        contours2, hierarchy2 = cv2.findContours(edges2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        #contours2, hierarchy2 = cv2.findContours(edges2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        cv2.drawContours(bgr_img1, contours1, -1, (0, 255, 0), 1)
        cv2.drawContours(bgr_img2, contours2, -1, (0, 255, 0), 1)
        
        for ic, contour in enumerate(contours1):
            
            try:
            
                feats = get_contourail_features(contour,gray_img1,img_uint1,gtruth_masks[0],bgr_img1)

                for fi,feat_name in enumerate(feats_list):
                    feats_dict[feat_name].append(feats[fi])
            except:
                print('exception',contour)

        for ic, contour in enumerate(contours2):
            
            try:
            
                feats = get_contourail_features(contour,gray_img2,img_uint2,gtruth_masks[1],bgr_img2)

                for fi,feat_name in enumerate(feats_list):
                    feats_dict[feat_name].append(feats[fi])
                    
            except:
                print('exception',contour)
        
        
        displayImageSRGB(img_uint1,im_id1)
        displayImageGreyS( gray_img1,'gray_img1')
        displayImageGreyS( pred_img1,'predmask 0')
        
        displayImageGreyS( pred_img1F,'predmaskF 0')
        
        displayImageGreyS((predicted_mask_with_threshold[0,:,:]*255).astype(np.uint8),'imi1')
        #print( np.min((predicted_mask[0, 0, :, :]*255).astype(np.uint8)),np.max((predicted_mask[0, 0, :, :]*255).astype(np.uint8)) )
        displayImageSBGR(bgr_img1,'bgr_img1')
        print('contours',len(contours1))
        displayImageGreyS( (gtruth_masks[0]*255).astype(np.uint8),'gtruth 1')
        
        #print('G',np.min(gtruth_masks[0]),np.max(gtruth_masks[0]))
        
        displayImageSRGB(img_uint2,im_id2)
        displayImageGreyS( gray_img2,'gray_img2')
        displayImageGreyS( pred_img2,'predmask 1')
        
        displayImageGreyS( pred_img2F,'predmaskF 1')
        
        displayImageGreyS((predicted_mask_with_threshold[1,:,:]*255).astype(np.uint8),'imi2')
        displayImageSBGR(bgr_img2,'bgr_img2')
        print('contours2',len(contours2))
        displayImageGreyS( (gtruth_masks[1]*255).astype(np.uint8),'gtruth 2')
"""

"\nimport matplotlib.pyplot as plt\nmodel.eval()\n\nfeats_dict = dict()\nfor feat in feats_list:\n    feats_dict[feat] = []\n\ndef displayImageGreyS(imgI,imgname):\n    if imgI.shape[0]>0:\n        f = plt.figure()\n        f.set_figwidth(3)\n        f.set_figheight(3)\n        plt.imshow( imgI, cmap='gray' )\n        plt.title(imgname)\n        plt.show()\n\nii = 0\n#20\n#while ii < len(lstdir)-2:\nwhile ii < 20:\n    print('ii',ii)\n    #print(ii,'/',len(lstdir)-2)\n    data_rootV = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation/'\n    filenamesV = lstdir[ii:ii+2]\n    ii += 2\n    val_df = pd.DataFrame(filenamesV, columns=['record_id'])\n    val_df['path'] = data_rootV + val_df['record_id'].astype(str)\n    val_ds = ContrailsDatasetPLV(\n        val_df,\n        image_size,\n        train = False\n    )\n    \n    gtruth_masks = []\n    \n    for nn in range(len(val_df)) :\n        sp = val_df['path'].iloc[nn]\n        #print(sp)\n        #print(s

In [23]:
submission = pd.read_csv('/kaggle/input/google-research-identify-contrails-reduce-global-warming/sample_submission.csv', index_col='record_id')


for i, data in enumerate(test_dl):
    images, image_id = data

    #print(image_id,image_id.shape)
    
    images = images.to(device)
    with torch.no_grad():
        predicted_mask = model.forward(images[:, :, :, :])
    if config["model"]["image_size"] != 256:
        predicted_mask = torch.nn.functional.interpolate(predicted_mask, size=256, mode='bilinear')
    predicted_mask = torch.sigmoid(predicted_mask).cpu().detach().numpy()
    
    predicted_mask_with_threshold = np.zeros((images.shape[0], 256, 256))
    #predicted_mask_with_threshold[predicted_mask[:, 0, :, :] < 0.5] = 0
    #predicted_mask_with_threshold[predicted_mask[:, 0, :, :] > 0.5] = 1
    
    predicted_mask_with_threshold[predicted_mask[:, 0, :, :] < 0.5] = 0
    predicted_mask_with_threshold[predicted_mask[:, 0, :, :] > 0.5] = 1
    
    #print(predicted_mask.shape,predicted_mask_with_threshold.shape)

    #print(images.shape[0])
    for img_num in range(0, images.shape[0]):
        
        image_idnum = str(image_id[img_num].cpu().detach().numpy())
        
        #print(image_idnum)
        
        false_color,rgb_images = get_id_imagesTest(image_idnum)
        img = rgb_images[4]
        img_uint1 = (img*255).astype(np.uint8)
        gray_img1 = cv2.cvtColor(img_uint1, cv2.COLOR_BGR2GRAY)
        
        #displayImageSRGB(img_uint1,image_idnum)
        
        current_mask = predicted_mask_with_threshold[img_num, :, :]
        
        pred_img1 = (current_mask*255).astype(np.uint8)
        
        #print(np.min(pred_img1),np.max(pred_img1))
        
        edges1 = (current_mask*255).astype(np.uint8)
        
        contours1, hierarchy1 = cv2.findContours(edges1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        #get_contourail_features(contour,gray_img,img_uint)
        
        #displayImageGreyS( pred_img1,'pred_img1')
        
        #print('len contours',len(contours1))
        
        for ic, contour in enumerate(contours1):
            #print(contour)
            """
            pred = 0
            try:
                feats = get_contourail_features(contour,gray_img1,img_uint1)
                feat_dict = dict()
                for fi,feat_name in enumerate(feats_list):
                    feat_dict[feat_name] = feats[fi]
                #print(feat_dict)
                featvec = []
                for ftc in featcols:
                    featvec.append(  feat_dict[ftc] )
                
                featvec = np.array(featvec)
                
                pred = modelML.predict_proba(featvec)[1]
                if pred > 0.13:
                    pred = 1
                else:
                    pred = 0
                
                #pred = int(modelML.predict(featvec))
                
            except:
                pass
            
            #print('pred',pred)
            """
            pred = 1
            
            if pred==0:
                cv2.drawContours(pred_img1, [contour], 0, (0,0,0) , -1)
                
        #displayImageGreyS( pred_img1,'pred_img1')
        
        current_mask = (pred_img1).astype(np.float64) / 255.0
        
        ###################################
        #print(current_mask.shape,type(current_mask[0,0]),np.min(current_mask),np.max(current_mask) )
        
        #current_mask_uint = (current_mask*255).astype(np.uint8)
        
        #kernel = np.ones((3,3), np.uint8) #3,5
        #current_mask_uint = cv2.dilate(current_mask_uint, kernel, iterations=1)
        
        #kernel = np.ones((3,3), np.uint8) #3,5
        #current_mask_uint = cv2.erode(current_mask_uint, kernel, iterations=1)
        
        #current_mask = (current_mask_uint).astype(np.float64) / 255.0
        
        ##################################
        
        current_image_id = image_id[img_num].item()
        
        submission.loc[int(current_image_id), 'encoded_pixels'] = list_to_string(rle_encode(current_mask))

In [24]:
"""
submission = pd.read_csv('/kaggle/input/google-research-identify-contrails-reduce-global-warming/sample_submission.csv', index_col='record_id')

for i, data in enumerate(test_dl):
    images, image_id = data
    
    images = images.to(device)
    with torch.no_grad():
        predicted_mask = model.forward(images[:, :, :, :])
    if config["model"]["image_size"] != 256:
        predicted_mask = torch.nn.functional.interpolate(predicted_mask, size=256, mode='bilinear')
    predicted_mask = torch.sigmoid(predicted_mask).cpu().detach().numpy()
    
    predicted_mask_with_threshold = np.zeros((images.shape[0], 256, 256))
    predicted_mask_with_threshold[predicted_mask[:, 0, :, :] < 0.5] = 0
    predicted_mask_with_threshold[predicted_mask[:, 0, :, :] > 0.5] = 1
    
    for img_num in range(0, images.shape[0]):
        current_mask = predicted_mask_with_threshold[img_num, :, :]
        current_image_id = image_id[img_num].item()
        
        submission.loc[int(current_image_id), 'encoded_pixels'] = list_to_string(rle_encode(current_mask))
        
"""

'\nsubmission = pd.read_csv(\'/kaggle/input/google-research-identify-contrails-reduce-global-warming/sample_submission.csv\', index_col=\'record_id\')\n\nfor i, data in enumerate(test_dl):\n    images, image_id = data\n    \n    images = images.to(device)\n    with torch.no_grad():\n        predicted_mask = model.forward(images[:, :, :, :])\n    if config["model"]["image_size"] != 256:\n        predicted_mask = torch.nn.functional.interpolate(predicted_mask, size=256, mode=\'bilinear\')\n    predicted_mask = torch.sigmoid(predicted_mask).cpu().detach().numpy()\n    \n    predicted_mask_with_threshold = np.zeros((images.shape[0], 256, 256))\n    predicted_mask_with_threshold[predicted_mask[:, 0, :, :] < 0.5] = 0\n    predicted_mask_with_threshold[predicted_mask[:, 0, :, :] > 0.5] = 1\n    \n    for img_num in range(0, images.shape[0]):\n        current_mask = predicted_mask_with_threshold[img_num, :, :]\n        current_image_id = image_id[img_num].item()\n        \n        submission.l

In [25]:
submission

,encoded_pixels
record_id,
1000834164244036115,-
1002653297254493116,-


In [26]:
submission.to_csv('submission.csv')